In [86]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from tensorflow import  keras
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [28]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14765487351803398170
xla_global_id: -1
]


In [29]:
print(tf.config.list_physical_devices('GPU'))

[]


In [87]:
heart = pd.read_csv('./Data/heart_df.csv')

In [88]:
heart

,state,general_health,physical_health,mental_health,health_insurance,health_care_doctors,no_doc_bc_cost,last_checkup,excercise_30,sleep,...,income_level,weight_kg,height_m,difficulty_walking,smoke100_lifetime,smokeless_tobacco_products,alcohol_consumption_30,high_risk_situations,ecigaret,heart_disease
0,1.0,2.0,3.0,30.0,2.0,3.0,1.0,4.0,1.0,5.0,...,1.0,48.0,170.0,2.0,1.0,3.0,0.0,2.0,1.0,0.0
1,1.0,3.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,7.0,...,99.0,NaN,163.0,2.0,NaN,NaN,NaN,NaN,NaN,0.0
2,1.0,3.0,0.0,0.0,1.0,1.0,2.0,1.0,1.0,7.0,...,7.0,NaN,173.0,2.0,2.0,3.0,0.0,2.0,2.0,0.0
3,1.0,2.0,0.0,0.0,1.0,1.0,2.0,1.0,1.0,7.0,...,NaN,57.0,168.0,2.0,2.0,3.0,0.0,2.0,2.0,0.0
4,1.0,4.0,20.0,30.0,1.0,1.0,2.0,2.0,1.0,8.0,...,5.0,82.0,175.0,2.0,1.0,1.0,0.0,2.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331523,72.0,3.0,0.0,0.0,2.0,1.0,2.0,2.0,1.0,6.0,...,99.0,55.0,150.0,2.0,2.0,3.0,0.0,2.0,NaN,0.0
331524,72.0,3.0,0.0,0.0,1.0,1.0,2.0,3.0,2.0,12.0,...,4.0,76.0,152.0,2.0,2.0,3.0,0.0,2.0,NaN,0.0
331525,72.0,3.0,0.0,0.0,1.0,1.0,2.0,2.0,1.0,8.0,...,1.0,72.0,124.0,2.0,2.0,3.0,0.0,2.0,NaN,0.0
331526,72.0,3.0,0.0,0.0,1.0,1.0,2.0,1.0,1.0,6.0,...,99.0,80.0,173.0,2.0,7.0,3.0,4.0,2.0,NaN,0.0


In [32]:
heart.drop(columns = 'Unnamed: 0', inplace = True)

In [33]:
heart.drop(columns = ['education_lvl', 'income_level', 'employment_status', 'rent_own', 'health_care_doctors','no_doc_bc_cost', 'smokeless_tobacco_products', 'high_risk_situations', 'ecigaret', 'state'], inplace = True)

In [34]:
heart.dtypes

state                     float64
general_health            float64
physical_health           float64
mental_health             float64
health_insurance          float64
last_checkup              float64
excercise_30              float64
sleep                     float64
stroke                    float64
asthma                    float64
skin_cancer               float64
other_cancer              float64
copd_type_issue           float64
arthritis_anyform         float64
depressive_disorder       float64
kidney_disease            float64
diabetes                  float64
weight_kg                 float64
height_m                  float64
difficulty_walking        float64
smoke100_lifetime         float64
alcohol_consumption_30    float64
heart_disease             float64
dtype: object

# Train-test-validation split

In [38]:
heart

,general_health,physical_health,mental_health,health_insurance,last_checkup,excercise_30,sleep,stroke,asthma,skin_cancer,...,copd_type_issue,arthritis_anyform,depressive_disorder,kidney_disease,diabetes,weight_kg,height_m,difficulty_walking,smoke100_lifetime,alcohol_consumption_30
0,2.0,3.0,30.0,2.0,4.0,1.0,5.0,2.0,1.0,1.0,...,1.0,1.0,1.0,2.0,1.0,48.0,170.0,2.0,1.0,0.0
1,3.0,0.0,0.0,1.0,1.0,1.0,7.0,2.0,1.0,2.0,...,2.0,1.0,1.0,2.0,3.0,NaN,163.0,2.0,NaN,NaN
2,3.0,0.0,0.0,1.0,1.0,1.0,7.0,2.0,2.0,2.0,...,2.0,1.0,2.0,2.0,3.0,NaN,173.0,2.0,2.0,0.0
3,2.0,0.0,0.0,1.0,1.0,1.0,7.0,1.0,2.0,2.0,...,2.0,2.0,2.0,2.0,3.0,57.0,168.0,2.0,2.0,0.0
4,4.0,20.0,30.0,1.0,2.0,1.0,8.0,2.0,1.0,2.0,...,1.0,1.0,2.0,2.0,1.0,82.0,175.0,2.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331523,3.0,0.0,0.0,2.0,2.0,1.0,6.0,2.0,2.0,2.0,...,2.0,2.0,1.0,2.0,3.0,55.0,150.0,2.0,2.0,0.0
331524,3.0,0.0,0.0,1.0,3.0,2.0,12.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,3.0,76.0,152.0,2.0,2.0,0.0
331525,3.0,0.0,0.0,1.0,2.0,1.0,8.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,3.0,72.0,124.0,2.0,2.0,0.0
331526,3.0,0.0,0.0,1.0,1.0,1.0,6.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,3.0,80.0,173.0,2.0,7.0,4.0


In [ ]:
target = heart.pop('heart_disease')

### Temporary filling nulls to test to see if everything works

In [39]:
heart.fillna(1, inplace = True)

# Preprocessing and Pipeline Building

In [40]:
train, val, test = np.split(target.sample(frac=1), [int(0.6*len(target)), int(0.8*len(target))])

In [41]:
print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

198916 training examples
66306 validation examples
66306 test examples


In [42]:
#creating list of columns names that needs to be either scaled or OHE
continous = ['physical_health', 'mental_health', 'last_checkup' , 'excercise_30', 'sleep', 'weight_kg',
             'height_m', 'alcohol_consumption_30']

categorical = list(heart.columns.drop(continous))

In [43]:
train

158521    0.0
234402    1.0
96015     0.0
34        0.0
243362    0.0
         ... 
180922    0.0
258355    0.0
237796    0.0
253112    0.0
310884    0.0
Name: heart_disease, Length: 198916, dtype: float64

In [45]:
# help to making input pipeline
def df_to_dataset(dataframe, shuffle=True, batch_size=200):
    df = heart.copy()
    labels = target
    df = {key: value[:,tf.newaxis] for key, value in heart.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(heart))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [46]:
batch_size = 20
train_ds = df_to_dataset(train, batch_size=batch_size)

C:\Users\xiao\AppData\Local\Temp/ipykernel_2296/3061089781.py:5: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in heart.items()}


In [47]:
#confirms that it works
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of sleep:', train_features['sleep'])
print('A batch of targets:', label_batch )

Every feature: ['general_health', 'physical_health', 'mental_health', 'health_insurance', 'last_checkup', 'excercise_30', 'sleep', 'stroke', 'asthma', 'skin_cancer', 'other_cancer', 'copd_type_issue', 'arthritis_anyform', 'depressive_disorder', 'kidney_disease', 'diabetes', 'weight_kg', 'height_m', 'difficulty_walking', 'smoke100_lifetime', 'alcohol_consumption_30']
A batch of sleep: tf.Tensor(
[[7.]
 [7.]
 [7.]
 [8.]
 [8.]
 [7.]
 [8.]
 [7.]
 [8.]
 [8.]
 [8.]
 [7.]
 [7.]
 [7.]
 [7.]
 [7.]
 [7.]
 [6.]
 [8.]
 [8.]], shape=(20, 1), dtype=float64)
A batch of targets: tf.Tensor([0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0.], shape=(20,), dtype=float64)


# Functions

In [48]:
# function that creates the layer for each column for the CONTINUOUS features
def get_normalization_layer(name,dataset):
    '''
    creates a layer for each continous feature as they each need to scaled and be their own layer
    '''
    # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

    # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the statistics of the data.
    normalizer.adapt(feature_ds)

    return normalizer

In [49]:
testing_standarization = train_features['physical_health']
layer = get_normalization_layer('physical_health', train_ds)
layer(testing_standarization)

<tf.Tensor: shape=(20, 1), dtype=float32, numpy=
array([[-0.38896462],
       [-0.38896462],
       [-0.38896462],
       [-0.28415087],
       [-0.28415087],
       [-0.38896462],
       [-0.38896462],
       [-0.38896462],
       [ 2.7554474 ],
       [-0.38896462],
       [-0.38896462],
       [-0.38896462],
       [-0.38896462],
       [-0.38896462],
       [-0.38896462],
       [ 1.7073101 ],
       [-0.38896462],
       [-0.38896462],
       [-0.38896462],
       [-0.38896462]], dtype=float32)>

#### Dealing with the categorical data

In [50]:
#creates the categorical layers for each column
def get_category_encoding_layer(name, dataset,  max_tokens=None):
    index = layers.IntegerLookup(max_tokens=max_tokens)

    # Prepare a `tf.data.Dataset` that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the set of possible values and assign them a fixed integer index.
    index.adapt(feature_ds)

    # Encode the integer indices.
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

    # Apply multi-hot encoding to the indices. The lambda function captures the
    # layer, so you can use them, or include them in the Keras Functional model later.
    return lambda feature: encoder(index(feature))

In [51]:
get_normalization_layer('general_health', train_ds)

# working on our actual data instead of testing out part on these functions

In [52]:
all_inputs = []
encoded_features = []

#  Dealing with the continuous numbers
for column_name in continous:
    numeric_col = tf.keras.Input(shape=(1,), name=column_name)
    normalization_layer = get_normalization_layer(column_name, train_ds)
    encoded_numeric_col = normalization_layer(numeric_col)
    all_inputs.append(numeric_col)
    encoded_features.append(encoded_numeric_col)

In [53]:
for column_name in categorical:
    categorical_col = tf.keras.Input(shape=(1,), name=column_name)
    encoding_layer = get_category_encoding_layer(column_name,train_ds)
    encoded_categorical_col = encoding_layer(categorical_col)
    all_inputs.append(categorical_col)
    encoded_features.append(encoded_categorical_col)

In [77]:
encoded_features

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_9')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_10')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_11')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_12')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_13')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_14')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_15')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_16')>,
 <KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'category_encoding')>,
 <KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'category_encoding_1')>,
 <KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'category_encoding_2')>,
 <KerasTensor: shape=(None, 3) dtype=float32 (created by la

In [54]:
all_inputs

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'physical_health')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'mental_health')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'last_checkup')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'excercise_30')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'sleep')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'weight_kg')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'height_m')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'alcohol_consumption_30')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'general_health')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'health_insurance')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'stroke')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'asthma')>,
 <KerasTensor: shape=(None, 1) dty

In [71]:
train_ds

<PrefetchDataset element_spec=({'general_health': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None), 'physical_health': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None), 'mental_health': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None), 'health_insurance': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None), 'last_checkup': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None), 'excercise_30': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None), 'sleep': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None), 'stroke': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None), 'asthma': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None), 'skin_cancer': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None), 'other_cancer': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None), 'copd_type_issue': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None), 'arthritis_anyform': TensorSpec(shape=(None, 1), dtype=tf.float64, name=None), 'depress

In [75]:
train

158521    0.0
234402    1.0
96015     0.0
34        0.0
243362    0.0
         ... 
180922    0.0
258355    0.0
237796    0.0
253112    0.0
310884    0.0
Name: heart_disease, Length: 198916, dtype: float64

In [76]:
all_features

<KerasTensor: shape=(None, 53) dtype=float32 (created by layer 'concatenate_4')>

In [ ]:
batch_size = 2000
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [82]:
#uses functional keras API
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(53, activation="relu")(all_features)
x = tf.keras.layers.Dense(40, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1, activation= 'sigmoid')(x)

model = tf.keras.Model(all_inputs, output)

In [83]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=METRICS)

In [84]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10


C:\Users\xiao\anaconda3\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


166/166 [==============================] - 14s 56ms/step - loss: 0.2984 - tp: 1858.0000 - fp: 5461.0000 - tn: 601519.0000 - fn: 54218.0000 - accuracy: 0.9100 - precision: 0.2539 - recall: 0.0331 - auc: 0.7449 - prc: 0.1947 - val_loss: 0.2412 - val_tp: 341.0000 - val_fp: 241.0000 - val_tn: 303249.0000 - val_fn: 27697.0000 - val_accuracy: 0.9157 - val_precision: 0.5859 - val_recall: 0.0122 - val_auc: 0.7994 - val_prc: 0.2929
Epoch 2/10
166/166 [==============================] - 12s 54ms/step - loss: 0.2479 - tp: 761.0000 - fp: 614.0000 - tn: 302876.0000 - fn: 27277.0000 - accuracy: 0.9159 - precision: 0.5535 - recall: 0.0271 - auc: 0.7818 - prc: 0.2811 - val_loss: 0.2379 - val_tp: 812.0000 - val_fp: 469.0000 - val_tn: 303021.0000 - val_fn: 27226.0000 - val_accuracy: 0.9165 - val_precision: 0.6339 - val_recall: 0.0290 - val_auc: 0.8072 - val_prc: 0.3093
Epoch 3/10
166/166 [==============================] - 12s 55ms/step - loss: 0.2458 - tp: 1028.0000 - fp: 768.0000 - tn: 302722.0000 - fn:

In [85]:
model.evaluate(test_ds)

166/166 [==============================] - 3s 16ms/step - loss: 0.2357 - tp: 1609.0000 - fp: 1033.0000 - tn: 302457.0000 - fn: 26429.0000 - accuracy: 0.9172 - precision: 0.6090 - recall: 0.0574 - auc: 0.8125 - prc: 0.3238


[0.2356855720281601,
 1609.0,
 1033.0,
 302457.0,
 26429.0,
 0.9171653389930725,
 0.6090083122253418,
 0.057386405766010284,
 0.8125298619270325,
 0.3238392472267151]

In [ ]:
# Sequential API
# model = tf.keras.Sequential([
#     '''Pipeline/concating needed here''',
#     tf.keras.layers.Dense(56, activation='relu'),
#     tf.keras.layers.Dense(30, activation='relu'),
#     tf.keras.layers.Dense(1)
#   ])
#
# model.compile(optimizer='adam',
#                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#                 metrics=METRICS)
# model.fit(train_ds, epochs=10, validation_data=val_ds)

In [58]:
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]


In [ ]:
# tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

C:\Users\xiao\AppData\Local\Temp/ipykernel_2296/3061089781.py:5: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in heart.items()}


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'assert_greater_equal/Assert/AssertGuard/Assert' defined at (most recent call last):
    File "C:\Users\xiao\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\xiao\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\xiao\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\xiao\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\xiao\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\xiao\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\xiao\anaconda3\lib\asyncio\base_events.py", line 596, in run_forever
      se

In [ ]:
# # Addition of the normalization layer into Sequential layer
# def get_basic_model():
#   model = tf.keras.Sequential([
#     normalizer,
#     tf.keras.layers.Dense(10, activation='relu'),
#     tf.keras.layers.Dense(10, activation='relu'),
#     tf.keras.layers.Dense(1)
#   ])
#
#   model.compile(optimizer='adam',
#                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#                 metrics=['accuracy'])
#   return model

In [ ]:
# model = get_basic_model()
# model.fit(continous_feat, target, epochs=15, batch_size=BATCH_SIZE)